In [1]:
!pip install --upgrade --quiet pip
!pip install --quiet farm-haystack[colab,faiss]==1.17.2
print('pip install haystack complete.')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scip

In [2]:
URL = 'https://www.intelligence.senate.gov/sites/default/files/publications/CRPT-113srpt288.pdf'
WORKING = '/kaggle/working/'

import os
import requests

# https://stackoverflow.com/a/56951135
def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file=file_path, mode='wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))


short_name = os.path.basename(URL)
if not os.path.exists(WORKING + short_name):
    download(url=URL, dest_folder=WORKING)
    print('downloaded {} to {}'.format(URL, WORKING))
else:
    print('file {} already exists.'.format(WORKING + short_name))
pdf_paths=[WORKING + short_name]

saving to /kaggle/working/CRPT-113srpt288.pdf
downloaded https://www.intelligence.senate.gov/sites/default/files/publications/CRPT-113srpt288.pdf to /kaggle/working/


In [3]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str='Flat')
print('created our document store')

created our document store


In [4]:
!apt-get install --quiet poppler-utils -y
print('pip install poppler utils complete.')
!sudo apt-get update
!sudo apt-get install --quiet poppler-utils -y
print('pip install poppler complete for real and for true')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 2562 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Ign:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
Ign:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.3
Err:2 http://security.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
  404  Not

In [5]:
# Initialize a PDFToTextConverter to convert the PDF to text
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PDFToTextConverter, PreProcessor, DensePassageRetriever
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
print('many imports complete.')

many imports complete.


In [6]:
#replace this paths with your input Data path

# Convert the PDF documents to Haystack-compatible format
pdf_converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=['en'])
pdf_docs = []

for pdf_path in pdf_paths:
    doc = pdf_converter.convert(file_path=pdf_path, meta=None)[0]
    pdf_docs.append(doc)
print('converted {} documents'.format(len(pdf_docs)))

pdftotext version 0.86.1
Copyright 2005-2020 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011 Glyph & Cog, LLC
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weigh

converted 1 documents


In [7]:
# Initializing the PreProcessor to clean and split the text
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by='word',
    split_length=100,
    split_respect_sentence_boundary=True,
)
processed_docs = preprocessor.process(pdf_docs)
print('document processing done.')

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

document processing done.


## Initializing the Retriever

We use a `DensePassageRetriever` and we invoke `update_embeddings` to index the embeddings of documents in the `FAISSDocumentStore`.

In [8]:
# Initialize a DensePassageRetriever
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='vblagoje/dpr-question_encoder-single-lfqa-wiki',
    passage_embedding_model='vblagoje/dpr-ctx_encoder-single-lfqa-wiki',
)
print('built retriever.')

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


built retriever.


In [9]:
document_store.write_documents(processed_docs)
document_store.update_embeddings(retriever)
print('updated embeddings.')

Writing Documents:   0%|          | 0/4841 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/4804 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/4816 [00:00<?, ? Docs/s]

updated embeddings.


In [10]:
from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path='vblagoje/bart_lfqa')
print('built generator.')

built generator.


In [11]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path='deepset/deberta-v3-large-squad2')
print('built the reader.')

built the reader.


In [12]:
from haystack.pipelines import GenerativeQAPipeline
from haystack.pipelines import ExtractiveQAPipeline

generative_pipeline = GenerativeQAPipeline(generator, retriever)
extractive_pipeline = ExtractiveQAPipeline(reader, retriever)

In [13]:
query = 'Where did the bulls live?'
result = extractive_pipeline.run(query=query, params={'Retriever': {'top_k': 5}, 'Reader': {'top_k': 1}})
print(result)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'Where did the bulls live?', 'no_ans_gap': -18.044864177703857, 'answers': [<Answer {'answer': "Shkai, Firas was living there. Prior to hearing about\nFiras' arrest, Ghul's understanding was that Firas was in Angorada", 'type': 'extractive', 'score': 0.00012068002251908183, 'context': "rst arrived in Shkai, Firas was living there. Prior to hearing about\nFiras' arrest, Ghul's understanding was that Firas was in Angorada with Khalid Ha", 'offsets_in_document': [{'start': 369, 'end': 489}], 'offsets_in_context': [{'start': 15, 'end': 135}], 'document_ids': ['1ce79791ac39a0708f4c0920e67b0896'], 'meta': {'_split_id': 2892, 'vector_id': '248'}}>], 'documents': [<Document: {'content': 'Ghul stated that he was not sure if Talha was a participant or\nsimply an observer. See 1679 I M111\nHassan Ghul was shown photos of individuals apprehendedby ( d ^ ^ M f c i on | October 2003 |\nand identified one as a Yemeni named Firas, "a well-trained fighter and experienced killer, who was known

In [14]:
print(result['answers'][0].answer)

Shkai, Firas was living there. Prior to hearing about
Firas' arrest, Ghul's understanding was that Firas was in Angorada


In [15]:
piperun=generative_pipeline.run(
    query='Where did the bulls live?', params={'Retriever': {'top_k': 5}} #output varies with different top_k values
)
for answer in piperun['answers']:
    print(answer.answer)

It depends on what you mean by "where did the bulls live." If you mean where they lived in the wild, then yes, they would have lived in a herd of cattle. If you are talking about domesticated animals, then the answer is probably somewhere in the middle.


In [16]:
piperun=generative_pipeline.run(
    query='What are Pointers in C++?', params={'Retriever': {'top_k': 5}}
)
piperun['answers'][0].answer

"Pointers in C++ are a way to represent the number of steps in a program. For example, let's say you have a program that needs to do something like this: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 90 95 96 97 98 99 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 115 116 116 117 118 119 120 123 124 125 126 127 128 128 128 129 129 128 129 128 128 131 129 129 130 131 132 133 134 135 135 136 137 148 149 150 151 152 152 153 154 155 156 157 157 162 162 163 164"